![](https://files.training.databricks.com/images/301/deployment_options_mllib.png)

There are four main deployment options:
* Batch pre-compute
* Structured streaming
* Low-latency model serving
* Mobile/embedded (outside scope of class)

We have already seen how to do batch predictions using Spark. Now let's look at how to make predictions on streaming data.

## ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) In this lesson you:<br>
 - Apply a SparkML model on a simulated stream of data

In [0]:
%run "./Includes/Classroom-Setup"

## Load in Model & Data

We are loading in a repartitioned version of our dataset (100 partitions instead of 4) to see more incremental progress of the streaming predictions.

In [0]:
from pyspark.ml.pipeline import PipelineModel

pipelinePath = "dbfs:/mnt/training/airbnb/sf-listings/models/sf-listings-2019-03-06/pipeline_model"
pipelineModel = PipelineModel.load(pipelinePath)

repartitionedPath =  "dbfs:/mnt/training/airbnb/sf-listings/sf-listings-2019-03-06-clean-100p.parquet/"
schema = spark.read.parquet(repartitionedPath).schema

## Simulate streaming data

**NOTE**: You must specify a schema when creating a streaming source DataFrame.

In [0]:
streamingData = (spark
                 .readStream
                 .schema(schema) # Can set the schema this way
                 .option("maxFilesPerTrigger", 1)
                 .parquet(repartitionedPath))

## Make Predictions

In [0]:
streamPred = pipelineModel.transform(streamingData)

Let's save our results.

In [0]:
import re

checkpointDir = userhome + "/machine-learning/stream_1p_checkpoint"
# Clear out the checkpointing directory
dbutils.fs.rm(checkpointDir, True) 

(streamPred
 .writeStream
 .format("memory")
 .option("checkpointLocation", checkpointDir)
 .outputMode("append")
 .queryName("pred_stream_1p")
 .start())

In [0]:
untilStreamIsReady("pred_stream_1p")

While this is running, take a look at the new Structured Streaming tab in the Spark UI.

In [0]:
display(
  sql("select * from pred_stream_1p")
)

In [0]:
display(
  sql("select count(*) from pred_stream_1p")
)

Now that we are done, make sure to stop the stream

In [0]:
for stream in spark.streams.active:
  print(f"Stopping {stream.name}")
  stream.stop() # Stop the stream

### What about Model Export?

* [MLeap](https://mleap-docs.combust.ml/)
  * Company that developed MLeap is no longer supporting it, and MLeap does not yet support Scala 2.12/Spark 3.0
* [ONNX](https://onnx.ai/)
  * ONNX is very popular in the deep learning community allowing developers to switch between libraries and languages, but only has experimental support for MLlib.
* DIY (Reimplement it yourself)
  * Error-prone, fragile
* 3rd party libraries
  * See XGBoost notebook
  * [H2O](https://www.h2o.ai/products/h2o-sparkling-water/)

### Low-Latency Serving Solutions

Low-latency serving can operate as quickly as tens to hundreds of milliseconds.  Custom solutions are normally backed by Docker and/or Flask (though Flask generally isn't recommended in production unless significant precations are taken).  Managed solutions also include:<br><br>

* [MLflow Model Serving (Preview)](https://databricks.com/blog/2020/06/25/announcing-mlflow-model-serving-on-databricks.html)
* [Azure Machine Learning](https://azure.microsoft.com/en-us/services/machine-learning/)
* [SageMaker](https://aws.amazon.com/sagemaker/)

-sandbox
&copy; 2020 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="http://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="http://help.databricks.com/">Support</a>